In [2]:
from selenium import webdriver
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import  WebDriverWait
import pandas as pd 
import time, threading, os, requests, ast

In [8]:
def scarpe(path):
    options = ChromeOptions()
    options.headless=True
    service = Service(ChromeDriverManager().install())
    driver = Chrome(service=service, options=options)
    driver.get(path)
    driver.maximize_window()
    
    
     # clck login fourbrotherstrading@icloud.com     
    try:
        # log_card =WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="site-nav__account"]'))) 
        login = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/a[text() = "Login"]')))
        if login:
            login.click()
    except Exception as e:
        print("no login")

    # get the username tab
    try:
        provided_u_name = "fourbrotherstrading@icloud.com"
        user_name = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.ID,'username')))   
        user_name.send_keys(provided_u_name)
    except Exception as e:
        print(f"No username tab found and the error is {e}")
    
    # get password tab
    try:
        provided_pass = "Muhssan7865"
        password = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.ID, 'password')))
        driver.execute_script("arguments[0].scrollIntoView();", password)
        password.send_keys(provided_pass)
    except Exception as e:
        print(f"No password tab found and the error is {e}") 
    
    # get the check done
    try:
        check = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, './/button[text() = "Sign in"]')))
        if check:
            # driver.execute_script("arguments[0].scrollIntoView();", check)
            check.click()
            # time.sleep(2)
            # for i in range(2):
            #     driver.back()
    except Exception as e:
        print(f"No check tab found and the error is {e}") 
    
     # number of cars
    try:
        # time.sleep(1)
        cars = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/span[@class="sort-page"]'))).text.strip()
        if cars:
            print(cars)
            splited_cars = cars.split(" ")
            total_cars = int(splited_cars[4])
            print(f"{total_cars} number of cars found")
        else:
            print("No cars found")
    except Exception as e:
        print(f"No cars found and error is {e}")
    
    
    # result
    results = []
    car_count = 0
    while car_count < total_cars: #total_cars
        
        try:
            page_cars = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/img[@class="card-img-top"]')))
            
            for i in range(len(page_cars)): # len(page_cars)
                if car_count >= total_cars:
                    break
                
                try:
                    page_cars = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/img[@class="card-img-top"]')))
                    driver.execute_script("arguments[0].scrollIntoView();", page_cars[i])
                    time.sleep(1)
                    page_cars[i].click()

                    details = {}
                    
                    # lot_num
                    try:
                        lot = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/span[@class="pill-item pill-item-lot"]'))).text.strip()
                        if lot:
                            details['Lot'] = lot
                        else:
                            details['Lot'] = 'na'
                    except Exception as e:
                        print("No lot values found")
                    
                    # time_date
                    try:
                        dt = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/span[@class="pill-item pill-item-time"]'))).text.strip()
                        if dt:
                            details['Start Time'] = dt.split(" ")[0]
                            details['Start Date'] = dt.split(" ")[1]
                        else:
                            details['Start Time'] = 'na'
                            details['Start Date'] = 'na'
                    except Exception as e:
                        print("No time and date")
                        
                    # veh cont --> done
                    try:
                        cont_card =WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="content-section content-section-white content-section-details"]'))) 
                        if cont_card:
                            # title and inspec repo --> done
                            try:
                                cont_head =WebDriverWait(cont_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="section-header"]')))  
                                if cont_head:
                                    title = cont_head.find_element(By.XPATH, './/div[@class="header-title"]').text.strip()
                                    if title:
                                        details['Title'] = title
                                    else:
                                        details['Title'] = 'na'
                                    
                                    inspec = cont_head.find_element(By.XPATH, './/a[text()="View PDF"]').get_attribute("href")
                                    if inspec:
                                        details['Inspection Report'] = inspec
                                    else:
                                        details['Inspection Report'] = 'na'
                                        
                            except Exception as e:
                                print("No cont head")
                            
                            # vendor  --> done
                            try:
                                vend =WebDriverWait(cont_card, 5).until(EC.presence_of_element_located((By.TAG_NAME, 'strong'))).text.strip()
                                if vend:
                                    splited_vend = vend.split(": ")
                                    details[splited_vend[0]] = splited_vend[1]
                                # else:
                            except Exception as e:
                                print("No vendor")
                            
                            # other det --> done
                            try:
                                cont_ul = WebDriverWait(cont_card, 5).until(EC.presence_of_element_located((By.XPATH, './/ul[@class="details-list"]')))
                                if cont_ul:
                                    cont_items = cont_ul.find_elements(By.XPATH, './/li[@class="detail-item"]')
                                    if cont_items:
                                        for cont_item in cont_items:
                                            cont_lbl = cont_item.find_element(By.TAG_NAME, 'span').text.strip()
                                            cont_val = cont_item.find_element(By.TAG_NAME, 'strong').text.strip()
                                            if cont_lbl and cont_val:
                                                details[cont_lbl] = cont_val
                                            else:
                                                print("No cont info found")
                                    else:
                                        print("No cont items")
                                else:
                                    print("No cont ul")
                            except Exception as e:
                                print("No cont info found")
                            
                            # features --> done
                            try:
                                fet =WebDriverWait(cont_card, 5).until(EC.presence_of_element_located((By.XPATH, './/p[@class="mt-4"]'))).text.strip()
                                if fet:
                                    details['Features'] = fet
                                else:
                                    details['Features'] = 'na'
                            except Exception as e:
                                print("No car features")  
                        else:
                            print("No cont header")
                    except Exception as e:
                        print("No veh cont information")
                    
                    
                    # images --> done
                    try:
                        imgs =WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/img[@alt="Vehicle image"]'))) 
                        if imgs:
                            imgs_list = [img.get_attribute('src') for img in imgs]
                            details['Images'] = ", ".join(imgs_list)
                        else:
                            print("No images found")
                        
                    except Exception as e:
                        print("No images")
                    
                    #details --> done
                    try:
                        car_det_card = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@id="economy-desktop"]')))
                        if car_det_card:
                            det_ul = car_det_card.find_element(By.XPATH, './/ul[@class="tab-grid details-list"]')
                            if det_ul:
                                det_items = det_ul.find_elements(By.XPATH, './/li[@class="detail-item"]')
                                for det_item in det_items:
                                    det_lbl = det_item.find_element(By.TAG_NAME, 'span').text.strip()
                                    det_val = det_item.find_element(By.TAG_NAME, 'strong').text.strip()
                                    if det_lbl and det_val:
                                        details[det_lbl]  = det_val
                                    else:
                                        print("No car details")            
                    except Exception as e:
                        print("No car details")
                        
                    
                    # mechanical
                    try:
                        mech_main = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@id="mechanical"]')))
                        if mech_main:
                            
                            # interior
                            try:
                                interior = WebDriverWait(mech_main, 5).until(EC.presence_of_element_located((By.XPATH, '(.//ul[@class="details-list"])[1]')))
                                if interior:
                                    int_items = interior.find_elements(By.XPATH, './/li[@class="detail-item"]')
                                    for int_item in int_items:
                                        int_lbl = int_item.find_element(By.TAG_NAME, 'span').text.strip()
                                        int_val = int_item.find_element(By.TAG_NAME, 'strong').text.strip()
                                        if int_lbl and int_val:
                                            details[int_lbl]  = int_val
                                        else:
                                            print("No car interior details")
                            except Exception as e:
                                print("No car interior info")
                                
                            # tyres
                            # try:
                            #     tyres = WebDriverWait(mech_main, 5).until(EC.presence_of_element_located((By.XPATH, '(.//ul[@class="details-list"])[2]')))
                            #     if tyres:
                            #         tyre_items = tyres.find_elements(By.XPATH, './/li[@class="detail-item"]')
                            #         for tyre_item in tyre_items:
                            #             tyre_lbl = tyre_item.find_element(By.TAG_NAME, 'span').text.strip()
                            #             tyre_val = tyre_item.find_element(By.TAG_NAME, 'strong').text.strip()
                            #             if tyre_lbl and tyre_val:
                            #                 details[tyre_lbl]  = tyre_val
                            #             else:
                            #                 print("No car tyres details")
                            # except Exception as e:
                            #     print("No car tyres info")
                            
                            # Tyres
                            # tyres
                            try:
                                tyre_main = WebDriverWait(mech_main, 5).until(EC.presence_of_element_located((By.XPATH, '(.//ul[@class="details-list"])[2]')))
                                if tyre_main:
                                    # tyre = WebDriverWait(tyre_main, 5).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="col-md-6"])[2]')))
                                    # interior
                                    # try:
                                        # tyre_inter = WebDriverWait(mech_main, 5).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="col-md-6"])[2]/table/tbody')))
                                    if tyre_main:
                                        tyre_trs = WebDriverWait(tyre_main, 5).until(EC.presence_of_all_elements_located((By.XPATH, '(.//li[@class="detail-item"])')))
                                        if tyre_trs:
                                            for tyre_tr in tyre_trs:
                                                tyre_lbl = tyre_tr.find_elements(By.TAG_NAME, "span")
                                                tyre_lbl2 = tyre_tr.find_element(By.TAG_NAME, "strong")
                                                if tyre_lbl:
                                                    details[tyre_lbl[0].text.strip()] = [tyre_lbl[1].text.strip(), tyre_lbl2.text.strip()]
                                                    
                                                # tyre_val = tyre_tr.find_element(By.TAG_NAME, "td").text.strip()
                                                # if tyre_lbl and tyre_val:
    
                                        else:
                                            print("No interior")
                                    # except Exception as e:
                                    #     print("No interior", e)
                            except Exception as e:
                                print(f"No tyre data")
                            
                            # damage images  --> done
                            try:
                                dmg_imgs =WebDriverWait(mech_main, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/img[@class="img-responsive largeimage"]'))) 
                                if dmg_imgs:
                                    dmg_imgs_list = [dmg_img.get_attribute('src') for dmg_img in dmg_imgs]
                                    details['Damaged_images'] = ", ".join(dmg_imgs_list)
                                else:
                                    details['Damaged_images'] = 'na'
                                
                            except Exception as e: 
                                print("No damage images found")
                            
                            # damage_text
                            try:
                                dmg_texts = WebDriverWait(mech_main, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/img[@class="img-responsive largeimage"]')))
                                if dmg_texts:
                                    dmg_text_list = [dmg_text.get_attribute("alt") for dmg_text in dmg_texts]
                                    details['Dmg text'] = ", ".join(dmg_text_list)
                                else:
                                    details['Dmg text'] = 'na'
                                
                            except Exception as e: 
                                print("No damage texts found")
                    except Exception as e:
                        print("No car mech information")
                    car_count+=1
                    results.append(details)
                    driver.back()   
                
                except Exception as e:
                    print("No link to click on")   
                
            if car_count % 30 == 0: # --> 25%25 == 0 and 50%25== 0
                try:
                    next_link = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/li[@class="page-item page-item-arrow page-item-arrow-next"]')))
                    if next_link:
                        driver.execute_script("arguments[0].scrollIntoView();", next_link)
                        time.sleep(2)
                        next_link.click()
                        
                        WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="card product-card"]')))
                        print("Hit the next button")
                except Exception as e:
                    print("No next found")
                    break
        except Exception as e:
            print("No more car links to click")
            break
    
    
    df = pd.DataFrame.from_dict(results)
    df.to_csv("protruck.csv", index=False)
    driver.quit()
    
path = 'https://www.protruckauctions.co.uk/auction/423'
scarpe(path)

1 - 30 of 62 Results
62 number of cars found
No vendor
No cont info found
No vendor
No cont info found
No tyre data
No vendor
No cont info found
No tyre data
No vendor
No cont info found
No car features
No tyre data
No vendor
No cont info found
No tyre data
No cont info found
No cont info found
No cont info found
No cont info found
No tyre data
No cont info found
No cont info found
No car features
No cont info found
No car features
No vendor
No cont info found
No cont info found
No tyre data
No cont info found
No tyre data
No cont info found
No car features
No tyre data
No cont info found
No car features
No tyre data
No tyre data
No cont info found
No tyre data
No cont info found
No cont info found
No tyre data
No cont info found
No car features
No tyre data
No cont info found
No cont info found
No car features
No cont info found
No tyre data
No cont info found
No cont info found
No cont info found
No cont info found
No tyre data
No cont info found
Hit the next button
No cont info foun

In [3]:
df = pd.read_csv("protruck.csv")
df['Miles'] = df['Miles'].apply(lambda x : x.split(" ")[0] if isinstance(x, str) and " " in x else "na")
df.rename(columns={"Registration":"Reg", "Registered":"D.O.R", "Miles":"Mileage", "Miles Warranted":"Mileage Warranted", "Fuel":"Fuel Type", "VAT":"VAT Status", "MOT Due":"MOT Expiry Date","MOT":"MOT Due", "Body Style":"Body Type", "Service documents":"Service Notes", "Last service date":"Last service", "Last service miles":"Last service mileage"}, inplace=True)
df['CC'] = df['CC'].apply(lambda x: f"{x/1000:.1f}"  if x else "na")
df['Lot'] = df['Lot'].apply(lambda x : x.split(" ")[-1] if isinstance(x, str) and " " in x else None)
df.drop(['Engine Size'],axis=1 , inplace=True)

info_cols = ['Carpet condition', 'Seat condition', 'Upholstery condition', 'Body Builder']
existing_cols = [col for col in info_cols if col in df.columns]
# Build the 'Additional Information' column
df['Additional Information'] = df[existing_cols].apply(lambda row: {col: row[col] if pd.notnull(row[col]) else 'na' for col in existing_cols},axis=1)
df.drop(columns=existing_cols, inplace=True)

df.to_csv("protruck_data.csv", index=False)
os.remove("protruck.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'protruck.csv'

### downloader

In [4]:
from PIL import Image, ImageDraw, ImageFont
def add_watermark_to_image(image_path, text="Sourced from ProTruck"):
    try:
        image = Image.open(image_path).convert("RGBA")
        txt_layer = Image.new("RGBA", image.size, (255, 255, 255, 0))
        draw = ImageDraw.Draw(txt_layer)

        # Load font
        try:
            font = ImageFont.truetype("arial.ttf", 16)
        except:
            font = ImageFont.load_default()

        # Calculate text size and position
        margin = 10
        bbox = draw.textbbox((0, 0), text, font=font)
        text_width = bbox[2] - bbox[0]
        text_height = bbox[3] - bbox[1]
        x = image.width - text_width - margin
        y = image.height - text_height - margin

        # Draw semi-transparent background box
        box_width = text_width + 2 * margin
        box_height = text_height + 2 * margin
        draw.rectangle([x - margin, y - margin, x - margin + box_width, y - margin + box_height],fill=(0, 0, 0, 160))

        # Draw watermark text
        draw.text((x, y), text, font=font, fill=(255, 255, 255, 200))

        # Merge and save
        watermarked = Image.alpha_composite(image, txt_layer).convert("RGB")
        watermarked.save(image_path)
        print(f"Watermark added to {image_path}")

    except Exception as e:
        print(f"Failed to watermark {image_path}: {e}")


In [ ]:
from urllib.parse import urlparse, urljoin
# urlparse: Parses a URL into components (scheme, netloc, path, etc.), making it easy to validate or extract information from the URL.
# urljoin: Joins a base URL with a relative path to form a complete URL.
df = pd.read_csv("protruck_data.csv")
report = df[['Reg', 'Inspection Report']]
reg_img = df[['Reg', 'Images']]
cond_imgs = df[['Dmg text', 'Damaged_images', 'Reg']]

def download_reports(data, main_folder="Inspec_report"): # here the main folder is Images
    
    # Create the main folder if it doesn't exist
    os.makedirs(main_folder, exist_ok=True)
    
    # loop around every row to get the info
    for index, row in data.iterrows():
        reg_no = row["Reg"] # separate reg nums
        report_urls = row["Inspection Report"]
        
        # Check if the inspection report is missing
        if not report_urls or pd.isna(report_urls):
            print(f"Missing Inspection Report Report of {reg_no}")
            
        else:
            if not report_urls.startswith(("http://", "https://")): # check if the url does not start with the values in the bracket
                report_urls = urljoin("https://", report_urls) # sets the urls starting from https://.....
            
            # parse the url
            parsed_url = urlparse(report_urls)
            
            # try downloading images
            try:
                # Download the image
                response = requests.get(report_urls, stream=True) # send the url for downloading
                response.raise_for_status() # Raises an exception if the HTTP request fails (e.g., 404 or 500).
                
                # Extracts the file name from the URL path (e.g., image.jpg from http://example.com/image.jpg).
                # If no file name is found, assigns a default name based on the index.
                file_name = os.path.basename(parsed_url.path) or f"inspec_repo.pdf"

                # Extracts the file extension (e.g., pdf). from the last index value
                file_extension = file_name.split(".")[-1]
                
                # Ensure the file has a valid extension
                if file_extension not in ["pdf", "jpeg", "png", "gif", "bmp", "webp"]:
                    file_name = f"inspec_repo.pdf" # set the extension if the extension is not in the above list
                
                # Construct the full path for the file
                full_file_name = os.path.join(main_folder, f"{reg_no}.pdf")
                
                # Save the image
                with open(full_file_name, 'wb') as f:
                    for chunk in response.iter_content(1024): # Reads the response in 1KB chunks to save memory.
                        f.write(chunk)
                
                print(f"Downloaded: {full_file_name}")
            except Exception as e:
                print(f"Failed to download {report_urls} for {reg_no}: {e}")

# images
def download_images(data, main_folder="Images"): # here the main folder is Images
    
    # Create the main folder if it doesn't exist
    os.makedirs(main_folder, exist_ok=True)
    
    # loop around every row to get the info
    for index, row in data.iterrows():
        reg_no = row["Reg"] # separate reg nums
        if pd.isna(row["Images"]) or not str(row["Images"]).strip():
            print(f"Skipping {reg_no} due to missing images.")
        else:
            image_urls = row["Images"].split(", ")
        
            # Create a subfolder for the car registration number
            reg_folder = os.path.join(main_folder, reg_no) # combine the main folder name and the reg num like Image/reg_no
            os.makedirs(reg_folder, exist_ok=True) # create the subfolder of the type mention in the above line of code
            
            # loop around the urls of the current row and also save index for further use
            for idx, url in enumerate(image_urls):
                url = url.strip()  # Remove extra spaces
                if not url.startswith(("http://", "https://")): # check if the url does not start with the values in the bracket
                    url = urljoin("https://", url) # sets the urls starting from https://.....
                
                # parse the url
                parsed_url = urlparse(url)

                # check if the parsed url is valid
                if not parsed_url.scheme or not parsed_url.netloc:
                    print(f"Invalid URL skipped: {url}") # incase some urls are incorret and are not loaded for downloading 
                    continue
                
                # try downloading images
                try:
                    # Download the image
                    response = requests.get(url, stream=True) # send the url for downloading
                    response.raise_for_status() # Raises an exception if the HTTP request fails (e.g., 404 or 500).
                    
                    # Extracts the file name from the URL path (e.g., image.jpg from http://example.com/image.jpg).
                    # If no file name is found, assigns a default name based on the index.
                    file_name = os.path.basename(parsed_url.path) or f"image_{idx + 1}.jpg"

                    # Extracts the file extension (e.g., jpg). from the last index value
                    file_extension = file_name.split(".")[-1]
                    
                    # Ensure the file has a valid extension
                    if file_extension not in ["jpg", "jpeg", "png", "gif", "bmp", "webp"]:
                        file_name = f"image_{idx + 1}.jpg" # set the extension if the extension is not in the above list
                    
                    # Construct the full path for the file
                    full_file_name = os.path.join(reg_folder, f"{reg_no}_{idx + 1}.jpg")
                    
                    # Save the image
                    with open(full_file_name, 'wb') as f:
                        for chunk in response.iter_content(1024):
                            f.write(chunk)

                    # Add watermark
                    add_watermark_to_image(full_file_name)
                    
                    print(f"Downloaded: {full_file_name}")
                except Exception as e:
                    print(f"Failed to download {url} for {reg_no}: {e}")

``
def download_images_damage(data, main_folder="Damage_203"):  # Main folder is now 'Damage'
    os.makedirs(main_folder, exist_ok=True)

    for index, row in data.iterrows():
        reg_no = row["Reg"]

        if pd.isna(row["Damaged_images"]) or not str(row["Damaged_images"]).strip():
            print(f"Skipping {reg_no} due to missing images.")
        elif pd.isna(row["Dmg_text"]) or not str(row["Dmg_text"]).strip():
            print(f"Skipping {reg_no} due to missing images.")
        else:
            dmg_texts = row["Dmg_text"].split(", ") 
            image_urls = row["Damaged_images"].split(", ") 

            reg_folder = os.path.join(main_folder, reg_no)  
            os.makedirs(reg_folder, exist_ok=True)  

            for idx, (url, dmg_text) in enumerate(zip(image_urls, dmg_texts)):
                url = url.strip()
                if not url.startswith(("http://", "https://")):
                    url = urljoin("https://", url)

                parsed_url = urlparse(url)
                if not parsed_url.scheme or not parsed_url.netloc:
                    print(f"Invalid URL skipped: {url}")
                    continue

                try:
                    response = requests.get(url, stream=True)
                    response.raise_for_status()

                    file_extension = os.path.splitext(parsed_url.path)[-1].lower()
                    if file_extension not in [".jpg", ".jpeg", ".png", ".gif", ".bmp", ".webp"]:
                        file_extension = ".jpg"

                    # Clean the damage text to make it a valid filename
                    safe_dmg_text = "".join(c if c.isalnum() or c in " _-" else "_" for c in dmg_text).strip()

                    full_file_name = os.path.join(reg_folder, f"{safe_dmg_text}_{idx + 1}_{file_extension}")

                    with open(full_file_name, 'wb') as f:
                        for chunk in response.iter_content(1024):
                            f.write(chunk)

                    # Add watermark
                    add_watermark_to_image(full_file_name)

                    print(f"Downloaded: {full_file_name}")
                except Exception as e:
                    print(f"Failed to download {url} for {reg_no}: {e}")

def start_funcs():
    thread1 = threading.Thread(target=download_reports, args=(report, ))
    thread2 = threading.Thread(target=download_images, args=(reg_img, ))
    thread3 = threading.Thread(target=download_images_damage, args=(cond_imgs, ))          
    
    thread1.start()
    thread2.start()
    thread3.start()              
    
    thread1.join()
    thread2.join()
    thread3.join()

if __name__ == "__main__":
    start_funcs()


Exception in thread Thread-5 (download_images_damage):
Traceback (most recent call last):
  File "c:\Users\HP\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandas\core\indexes\base.py", line 3805, in get_loc
    return self._engine.get_loc(casted_key)
           ~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
  File "index.pyx", line 167, in pandas._libs.index.IndexEngine.get_loc
  File "index.pyx", line 196, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\\_libs\\hashtable_class_helper.pxi", line 7081, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\\_libs\\hashtable_class_helper.pxi", line 7089, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'Dmg_text'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\HP\AppData\Local\Programs\Python\Python313\Lib\threading.py", line 1041, in _bootstrap_inner
    self.run()
    ~~~~~~~~^^
  File "C:\Users\HP\AppData\Roaming\Python\Python

Downloaded: Inspec_report\YD60BVC.pdf
Watermark added to Images\YD60BVC\YD60BVC_1.jpg
Downloaded: Images\YD60BVC\YD60BVC_1.jpg
Downloaded: Inspec_report\MX13WBM.pdf
Watermark added to Images\YD60BVC\YD60BVC_2.jpg
Downloaded: Images\YD60BVC\YD60BVC_2.jpg
Watermark added to Images\YD60BVC\YD60BVC_3.jpg
Downloaded: Images\YD60BVC\YD60BVC_3.jpg
Downloaded: Inspec_report\WU55HGZ.pdf
Downloaded: Inspec_report\WU55HHA.pdf
Watermark added to Images\YD60BVC\YD60BVC_4.jpg
Downloaded: Images\YD60BVC\YD60BVC_4.jpg
Downloaded: Inspec_report\WV06GNO.pdf
Watermark added to Images\YD60BVC\YD60BVC_5.jpg
Downloaded: Images\YD60BVC\YD60BVC_5.jpg
Downloaded: Inspec_report\WO17PZV.pdf
Watermark added to Images\YD60BVC\YD60BVC_6.jpg
Downloaded: Images\YD60BVC\YD60BVC_6.jpg
Downloaded: Inspec_report\WM17EUK.pdf
Watermark added to Images\YD60BVC\YD60BVC_7.jpg
Downloaded: Images\YD60BVC\YD60BVC_7.jpg
Downloaded: Inspec_report\WO17RBZ.pdf
Downloaded: Inspec_report\WP69OZE.pdf
Watermark added to Images\YD60BVC\Y